# Setup

In [10]:
%pip install -r requirements.txt

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/15/64/db3462b358072387b8e93e6e6a38d3c741a17b4a84171ef01d6c85c63f25/openai-1.63.2-py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/46/eb/e7f063ad1fec6b3178a3cd82d1a3c4de82cccf283fc42746168188e1cdd5/anyio-4.8.0-py3-none-any.whl.metadata
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for httpx<1,>=0.23.

In [12]:
import requests
import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=api_key
)

# Gather Story Summary

## Wikipedia

In [5]:
def get_wikipedia_summary(title):
    """
    Fetches the first extract of a Wikipedia article using the public MediaWiki API.
    Returns a text summary or an empty string if not found.
    """
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "format": "json",
        "titles": title
    }
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        for page_id, page_content in pages.items():
            if "extract" in page_content:
                # This is a raw textual extract from Wikipedia
                return page_content["extract"]
    except requests.RequestException as e:
        print(f"[Wikipedia] Error fetching summary: {e}")

    return ""

In [8]:
# testing with my fav solarpunk novel
book = "The Windup Girl"
summary = get_wikipedia_summary(book)
print(summary)

The Windup Girl is a biopunk science fiction novel by American writer Paolo Bacigalupi. It was his debut novel and was published by Night Shade Books on September 1, 2009. The novel is set in a future Thailand and covers a number of contemporary issues such as global warming and biotechnology.
The Windup Girl was named as the ninth best fiction book of 2009 by Time magazine. It won the 2010 Nebula Award and the 2010 Hugo Award (tied with The City & the City by China Miéville), both for best novel. The book also won the 2010 Campbell Memorial Award, the 2010 Compton Crook Award and the 2010 Locus Award for best first novel.


== Setting ==
The Windup Girl is set in 23rd-century Thailand. Global warming has raised the levels of world's oceans, carbon fuel sources have become depleted, and manually wound springs are used as energy storage devices. Biotechnology is dominant and megacorporations (called calorie companies) like AgriGen, PurCal and RedStar control food production through "gen

# Pass Book Summary thru the First Agent

In [ ]:
def query_openai(system_prompt, user_prompt):
    """Send a prompt to OpenAI and return the response."""
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return completion.choices[0].message.content

In [ ]:
# Load the system prompt from system_prompts/story_analysis_prompt.md
with open("system_prompts/story_analysis_prompt.md", "r") as file:
    system_prompt = file.read()

# Generate a response using the OpenAI API

